# Introduction to Scientific Python 3

ICTP Summer School on Quantum Dynamics of Matter, Light and Information

Date: August 20, 2025

Lecturer: Chris Laumann


For this lecture we will use the inline backend instead of the notebook backend. The inline backend doesn't support interactive manipulation of plots like the notebook back end does, but sometimes it's simpler -- and it's not hard to switch if you prefer one or the other.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from numpy import log, sqrt, exp, r_, array

# Optimizing in Python #


## First rule of optimization: 

**Your development time is worth a lot more than CPU time.**

Make it work **simply and legibly** first. Write simple test cases to make sure your code is correct. If, and only if, it isn't fast enough to get to the sizes you need, then start optimizing.

## Second rule of optimization:

**Measure twice, cut once.**

Use profiling tools to identify the bottlenecks in your code. They may not be where you expect. The two most useful built in tools are **%timeit** and **%prun** (which calls a module called cProfile). 

## Third rule of optimization:

**Algorithms before constants.**

Make sure your algorithmic approach is asymptotically efficient before worrying about shaving off constant overheads. Going from $T\approx 10N$ to $T\approx5N$ is good, but $T \approx 5 N^2$ to $T \approx 10N$ is way better. 

## Practical Tips for Numerical Code:

**Vectorize Loops** Use slicing tricks (if possible) to avoid loops over numpy arrays. 

**Avoid accidental copies** Better to use views, not copies of big arrays. Change things in place if possible so no new allocation needs to happen. 

**Contiguous memory is faster** Modern CPUs cache blocks of data so indices being contracted (striding of data) can matter.

**Use high level toolkits** Scipy and numpy provide lots of common precompiled algorithms that are mature and optimized. EG. Use scipy.fftpack for fast fourier transforms and look through scipy.signal for signal processing. 

## Last Resort:

**Weave in compiled code** If some bottleneck just has to be faster, consider replacing that one function with compiled C or Cython (compiled Python). Probably try Cython first. Rule of thumb: keep your pure python version around and make sure your compiled and pythonic versions produce the same output on a collection of test cases (unit tests). 


# 2D Ising Model Monte Carlo

Let's implement a classical 2D Ising model Monte Carlo in Python.
We consider the reduced Hamiltonian ($\sigma_i = \pm 1$):

$$\beta H = - J \sum_{\langle i j \rangle} \sigma_i \sigma_j - H \sum_i \sigma_i$$

so that $J>0$ corresponds to ferromagnetism. For reference, on a square lattice, the critical coupling is known exactly (from duality or from Onsager's solution):

In [ ]:
Jcrit = log(1+sqrt(2.))/2

Let's define our couplings.

In [ ]:
Lx, Ly = 100, 100

J = 1. * Jcrit
H = 0.

It's natural to store the configuration in a 2D array. Let's use an integer array and store +-1. 

In [ ]:
sigma = np.ones((Lx,Ly), dtype='i4')

In [ ]:
plt.figure()
plt.imshow(sigma, vmin = -1, vmax = 1)

How to calculate energy?

In [ ]:
def energy(sigma):
    en = 0
    
    # horizontal bonds
    for i in range(Lx-1):
        for j in range(Ly):
            en += -J * sigma[i,j]*sigma[i+1,j]
            
    # vertical bonds
    for i in range(Lx):
        for j in range(Ly-1):
            en += -J * sigma[i,j]*sigma[i,j+1]
    
    # field
    for i in range(Lx):
        for j in range(Ly):
            en += -H * sigma[i,j]
    
    return en

In [ ]:
%timeit energy(sigma)

This is a bit long-winded and slow. First, vectorize any for loops you can!

In [ ]:
def energy(sigma):
    return (-J * np.sum(sigma[:-1,:] * sigma[1:,:]) 
         + -J * np.sum(sigma[:,:-1] * sigma[:,1:]) 
         + -H * np.sum(sigma))


In [ ]:
%timeit energy(sigma)

Faster and more compact! Let's write an update step.

In [ ]:
def mcflip(sigma):
    """Loop through sites and randomly flip with probability
    set by the relative Gibbs weight."""
    
    e0 = energy(sigma)

    for i in range(Lx):
        for j in range(Ly):
            # propose flip
            sigma[i,j] *= -1
            
            e1 = energy(sigma)
            
            deltaE = e1 - e0
            p = exp(- deltaE)/(exp(- deltaE) + exp(deltaE))
            
            if np.random.rand() < p:
                # accept flip and new energy
                e0 = e1
            else:
                # reject flip
                sigma[i,j] *= -1
                
    return
    

In [ ]:
mcflip(sigma)

In [ ]:
plt.figure(figsize=(3,3))
plt.imshow(sigma, vmin = -1, vmax = 1)

In [ ]:
%timeit mcflip(sigma)

Ack! Where is all the time going?

In [ ]:
%%prun
mcflip(sigma)

Energy takes time O(N) and we call it O(N) times. That's O(N*N). Bad!

We only need $\Delta E$ which is local. Indeed, we only need the coefficient of the $\sigma$ term we wish to flip in the energy in the fixed background of other spins.
Thus, let's calculate the local effective field.

In [ ]:
def mcflip(sigma):
    """Loop through sites and randomly flip with probability
    set by the relative Gibbs weight."""
    
    for i in range(Lx):
        for j in range(Ly):
            # calculate local field 
            localH = H
            
            if i < Lx-1:
                localH += J*sigma[i+1,j]
            if i > 0:
                localH += J*sigma[i-1,j]
            if j < Ly-1:
                localH += J*sigma[i,j+1]
            if j > 0:
                localH += J*sigma[i,j-1]
            
            # sample a spin configuration 
            m = np.tanh(localH)
            p = (m+1)/2
            
            sigma[i,j] = 2*(np.random.rand() < p) - 1
                
    return
    

In [ ]:
%timeit mcflip(sigma)

Algorithm first! This is now O(N) instead of O(N^2). Can we vectorize it? There are two tricky bits: 
1. The special cases on the boundary
2. We can only do a parallel update in a fixed background.

Answers:
1. Set the boundary to zero and only update the bulk
2. Vectorize over sublattices

In [ ]:
def setBoundary(sigma, val = 0):
    sigma[0,:]  = val
    sigma[-1,:] = val
    sigma[:,0]  = val
    sigma[:,-1] = val
    return

In [ ]:
setBoundary(sigma, 0)

In [ ]:
def mcflip_sublat(sigma,a,b):
    """Update on sublattices defined by and randomly flip with probability
    set by the relative Gibbs weight."""
    
    # localH acts on the bulk sublattice which is defined by the slice
    # sigma[1+a:-1:2, 1+b:-1:2]
    
    localH = H + J * (
            sigma[2+a:   :2, 1+b: -1:2] + # to the right
            sigma[  a: -2:2, 1+b: -1:2] + # to the left
            sigma[1+a: -1:2, 2+b:   :2] + # up
            sigma[1+a: -1:2,   b: -2:2]   # down
            )
    
    m = np.tanh(localH)
    p = (m+1)/2
    
    sigma[1+a:-1:2, 1+b:-1:2] = 2*(np.random.rand(Lx//2-1,Ly//2-1) < p) - 1
                
    return

def mcflip(sigma):
    mcflip_sublat(sigma,0,0)
    mcflip_sublat(sigma,1,0)
    mcflip_sublat(sigma,0,1)
    mcflip_sublat(sigma,1,1)
    return

In [ ]:
mcflip(sigma)

In [ ]:
%timeit mcflip(sigma)

Okay, that's pretty quick. Let's try animating.

In [ ]:
import matplotlib.animation

fig, ax = plt.subplots()
im = plt.imshow(sigma, vmin = -1, vmax = 1)

def animate(i):
    """Take an mcstep and update image."""
    mcflip(sigma)
    im.set_data(sigma)
    return

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=100, interval=500/30)
plt.show()

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

# TASKS #

1. Measure the correlation time of the energy at several $J$'s. See code below for help.
2. Extract the magnetization, energy, susceptibility, specific heat as a function of coupling $J$ across the transition at $J_c$ and plot it.
3. Do this as a function of system size $L$ to see sharpening at the transition.
4. Can you modify the code to use fixed rather than free boundary conditions? Periodic boundary conditions?

Don't forget that shift-tab can bring up help for anything under your cursor and that searching on the web for help is A-OK.

In [ ]:
def magnetization(sigma):
    return np.mean(sigma[1:-1,1:-1]) # exclude boundary spins

In [ ]:
def runSample(nSteps = 10000, subSample = 10):
    """Run nSteps Monte Carlo sweeps on the 2D Ising model.
    Does not initialize the sample so can be called sequentially without
    reinitialization if desired.
    
    Returns:
        histMag -- array of magnetizations at every subSample timesteps
        histEn  -- array of energies at every subSample timesteps
    """
    
    # create lists to grow
    histMag = []
    histEn = []
        
    for t in range(10000):
        mcflip(sigma)
        if t%subSample == 0:
            histMag.append(magnetization(sigma))
            histEn.append(energy(sigma))
    
    # return as arrays
    return (array(histMag), array(histEn))

Computing autocorrelations.

In [ ]:
J = 0.9*Jcrit
histMag, histEn = runSample(subSample=1)

In [ ]:
plt.plot(histMag)

In [ ]:
c = np.correlate(histMag-np.mean(histMag),histMag-np.mean(histMag), 'same')
# get positive time part only (b/c autocorrelation is t -> -t symmetric)
c = c[len(c)//2:]
plt.plot(c)
plt.xlabel(r'MC Time $t$')
plt.ylabel(r'Magnetization Autocorrelation $\langle m(t) m(0) \rangle$')

In [ ]:
fitToTime = 100
fitTs = np.r_[:fitToTime]

pf = np.polyfit(fitTs,log(c[:fitToTime]),1)
tau = -1/pf[0]

plt.semilogy(np.abs(c))
plt.semilogy(fitTs, exp(np.polyval(pf,fitTs)), '--', 
         label=r'$\ln \langle M(t)M(0)\rangle = - t / %1.1f$'%(tau))
plt.legend()

plt.xlim(0,500)

Collecting data across sweep in coupling $J$.

In [ ]:
Js = r_[0.5:1.5:21j] * Jcrit

mags    = np.zeros_like(Js)
magvars = np.zeros_like(Js)
ens     = np.zeros_like(Js)
envars  = np.zeros_like(Js)

for (i,J) in enumerate(Js):
    print("J: ", J)
    
    # reinitialize spin configuration
    sigma = np.ones_like(sigma)
    setBoundary(sigma, 0) # free boundary conditions

    (histMag, histEn) = runSample(nSteps=10000, subSample = 10)
    
    mags[i]    = np.mean(histMag)
    magvars[i] = np.var(histMag)
    ens[i]     = np.mean(histEn)
    envars[i]  = np.var(histEn)


In [ ]:
plt.figure()
plt.plot(Js,abs(mags))
plt.vlines([Jcrit], 0,1, linestyle='dashed')
plt.ylabel(r'$\langle m \rangle$')
plt.xlabel(r'$J$')